### Notebook which does some data spotchecking to ensure the data processing didn't introduce any errors

In [5]:
import xarray as xr
from joblib import Parallel, delayed
import pickle
import pandas as pd
from openavalancheproject.prep_ml import PrepML
import numpy as np

In [2]:
interpolate = 1 
data_root = '/media/scottcha/E1/Data/OAPMLData/'

In [3]:
pml = PrepML(data_root, interpolate, date_start='2018-11-01', date_end='2019-04-30', date_train_test_cutoff='2019-04-01')   

In [54]:
#get produced dataset
ml_path = data_root + '/5.MLData/'
num_features = 978
interpolation = 1
label = 'Day1DangerAboveTreeline'
file_label = 'day1above'
X = np.load(ml_path + '/X_all_' + file_label + '.npy', mmap_mode='r')



In [56]:
#read in the corresponding label files and concat them
#can get the right values here based on the contents of hte ml_path directory
num_y_batches = 4
num_y_train_files_per_batch = 10
num_y_test_batches = 1
num_y_test_files_per_batch = 10


y_df = None
for i in range(0, num_y_batches):
    for j in range(0, num_y_train_files_per_batch):        
        df = pd.read_parquet(ml_path + '/y_train_batch_' + str(i) + '_' + file_label + '_' + str(j) + '.0.parquet')  
        y_df = pd.concat([y_df, df])

for i in range(0, num_y_test_batches):
    for j in range(0, num_y_test_files_per_batch):
        df = pd.read_parquet(ml_path + '/y_test_batch_' + str(i) + '_' + file_label + '_' + str(j) + '.0.parquet')  
        y_df = pd.concat([y_df, df])

y_df = y_df.reset_index(drop=True)

In [57]:
num_files_to_validate = 10
y_to_validate = y_df.sample(n=num_files_to_validate)

In [58]:
def is_equal(a, b):
    return np.array_equiv(np.nan_to_num(a),np.nan_to_num(b))
    

In [59]:
features = pd.read_csv(ml_path + '/FeatureLabels.csv')['0']

In [60]:
features

0              UGRD_planetaryboundarylayer_min
1              VGRD_planetaryboundarylayer_min
2             VRATE_planetaryboundarylayer_min
3                             GUST_surface_min
4                                 HGT_10mb_min
                        ...                   
973     HGT_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
974    PRES_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
975    VWSH_PV_EQ_M2eM06_Km_2_kg_s_surface_avg
976                     PRMSL_meansealevel_avg
977                           N5WAVH_500mb_avg
Name: 0, Length: 978, dtype: object

In [61]:
for r in y_to_validate.iterrows():
    i = r[0]
    r = r[1]
    
    test_region = r['UnifiedRegion']
    test_lat = r['latitude']
    test_lon = r['longitude']
    test_lookback = 180
    test_date = r['parsed_date']
    print('Sample: ' + r['sample'])
    data = pml.get_data_zarr(test_region, lat=test_lat, lon=test_lon, lookback_days=test_lookback, date=test_date, variables=features.values)
    data = data.sortby('variable')
    data2 = X[i]
    assert is_equal(data.vars.values, data2)

Sample: 2018-02-16 00:00:00: WA Cascades East, South
*Opening file /media/scottcha/E1/Data/OAPMLData//4.GFSFiltered1xInterpolationZarr//17-18/Washington/Region_WA Cascades East, South.zarr
Sample: 2015-12-22 00:00:00: WA Cascades West, Central
*Opening file /media/scottcha/E1/Data/OAPMLData//4.GFSFiltered1xInterpolationZarr//15-16/Washington/Region_WA Cascades West, Central.zarr
Sample: 2015-12-26 00:00:00: WA Cascades West, South
*Opening file /media/scottcha/E1/Data/OAPMLData//4.GFSFiltered1xInterpolationZarr//15-16/Washington/Region_WA Cascades West, South.zarr
Sample: 2019-04-01 00:00:00: WA Cascades West, Central
*Opening file /media/scottcha/E1/Data/OAPMLData//4.GFSFiltered1xInterpolationZarr//18-19/Washington/Region_WA Cascades West, Central.zarr
Sample: 2016-02-21 00:00:00: WA Cascades West, Mt Baker
*Opening file /media/scottcha/E1/Data/OAPMLData//4.GFSFiltered1xInterpolationZarr//15-16/Washington/Region_WA Cascades West, Mt Baker.zarr
Sample: 2016-01-23 00:00:00: WA Cascades 

<xarray.Dataset>
Dimensions:    (time: 180, variable: 978)
Coordinates:
    latitude   float64 46.5
    longitude  float64 -121.2
  * variable   (variable) <U52 'ABSV_1000mb_avg' ... 'WILT_surface_min'
Dimensions without coordinates: time
Data variables:
    vars       (variable, time) float64 nan nan nan nan ... 0.0608 0.0608 0.0608

In [48]:
data2

memmap([[          nan,           nan,           nan, ..., 1.1742343e-04,
         1.1615803e-04, 8.2262974e-05],
        [          nan,           nan,           nan, ..., 1.4664771e-04,
         1.6374243e-04, 1.3292261e-04],
        [          nan,           nan,           nan, ..., 1.0645129e-04,
         9.8841061e-05, 6.1168212e-05],
        ...,
        [          nan,           nan,           nan, ..., 6.0799997e-02,
         6.0799997e-02, 6.0799997e-02],
        [          nan,           nan,           nan, ..., 6.0800001e-02,
         6.0800001e-02, 6.0800001e-02],
        [          nan,           nan,           nan, ..., 6.0800001e-02,
         6.0800001e-02, 6.0800001e-02]], dtype=float32)

In [11]:
#check data
X_df.vars.sel(variable='SNOD_surface_avg').isel(sample=1)

<xarray.DataArray 'vars' (time: 180)>
array([     nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan,      nan,      nan,      nan,      nan, 0.044298,
       0.034412, 0.047212, 0.050715, 0.087223, 0.075625, 0.061544, 0.052231,
       0.050212, 0.037768, 0.024764, 0.108682, 0.074953, 0.06105 , 0.046642,
       0.039942, 0.002136, 0.002187, 0.001866, 0.00126 ])
Coordinates:
    latitude   float64 ...
    longitude  float64 ...
    sample     <U34 '2018-11-20 00:00:00: Gunnison Zone'
    variable   <U52 'SNOD_surface_avg'
Dimensions without coordinates: time
Attributes:
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs

In [19]:
y_df.iloc[1]['Day1DangerAboveTreeline']

'Low'

In [20]:
X_df.vars.values[:,:,:].shape

(52, 1131, 180)

In [21]:
pd.DataFrame(X_df.vars.values[1,:,:].T).tail()

,0,1,2,3,4,5,6,7,8,9,...,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130
175,24100.000000,-0.168962,-1.753429,0.0,1.422541,46293.923992,249.204329,0.001400,74.593879,1.706825,...,0.004236,NaN,NaN,NaN,NaN,NaN,NaN,102090.727563,5651.811016,1.0
176,22996.259412,-0.523364,-1.119112,0.0,0.809715,46297.963806,247.508666,0.001596,73.996220,8.486597,...,0.005440,NaN,NaN,NaN,NaN,NaN,NaN,101739.476881,5646.625665,1.0
177,16542.822444,1.241429,-1.686924,0.0,1.345743,46435.123433,250.185911,0.001300,73.730263,12.006322,...,-0.001205,NaN,NaN,NaN,NaN,NaN,NaN,102157.798651,5660.990929,1.0
178,24100.000000,0.647589,-2.061763,0.0,2.073283,46354.168772,248.008666,0.001500,78.148723,15.496969,...,-0.002431,NaN,NaN,NaN,NaN,NaN,NaN,102336.770842,5673.786992,1.0
179,24100.000000,0.071916,-1.697279,0.0,1.506249,46309.318633,248.913022,0.001400,86.228267,9.082058,...,0.000104,NaN,NaN,NaN,NaN,NaN,NaN,102429.080032,5673.707198,1.0
